<a href="https://colab.research.google.com/github/leonardo69-code/Python---Banco-de-Dados/blob/main/SQL_II_A_Vingan%C3%A7a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 22.6 MB/s eta 0:00:00


In [ ]:
import mysql.connector
import pandas as pd

In [ ]:
host = '34.70.26.188' # endereço do banco externo que eu quero acessar
user = 'root'
password = ''
database = 'locadora'

In [ ]:
!curl ipecho.net/plain

34.106.97.48

In [ ]:
def consulta(query, tabela):
  connection = mysql.connector.connect(
      host = host,
      user = user,
      password = password,
      database = database
  )
  cursor = connection.cursor()

  try:
    cursor.execute(query)
    result = cursor.fetchall()
    globals()[tabela] = pd.DataFrame(result, columns=cursor.column_names)
    display(globals()[tabela])
  finally:
    cursor.close()
    connection.close()

In [ ]:
def ddl(comando):
    connection = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    cursor = connection.cursor()

    try:
        cursor.execute(comando)
        connection.commit()  # Confirma as mudanças no banco
        print(f"Comando DDL executado com sucesso:\n{comando}")
    except mysql.connector.Error as err:
        print(f"Erro ao executar o comando DDL: {err}")
    finally:
        cursor.close()
        connection.close()

# GROUP_CONCAT

In [ ]:
'''
Liste os estados onde os clientes alugaram carros e os nomes dos clientes de
cada estado.
'''
consulta('''
SELECT estado, nome
FROM cliente;
''',''' ''')

,estado,nome
0,RJ,Ana Silva
1,RJ,Bruna Pereira
2,RJ,Túlio Nascimento
3,SP,Fernando Souza
4,SP,Lúcia Andrade


In [ ]:
'''
No bloco anterior temos uma situação onde cada estado aparece mais
de uma vez, pois, cada estado está associado à mais de um cliente, nosso
objetivo é apresentar uma consulta, cujo conjunto-resultado traga duas colunas
sendo uma para estado e outra para os nomes dos clientes, no entanto não desejamos
repetição de linhas de estado. Assim , podemos utilizar o GROUP_CONCAT para
resolver essa situação.
'''
consulta('''
SELECT estado, GROUP_CONCAT(nome SEPARATOR ', ') AS nomes
FROM cliente
GROUP BY estado;
''',''' ''')

,estado,nomes
0,RJ,"Ana Silva, Bruna Pereira, Túlio Nascimento"
1,SP,"Fernando Souza, Lúcia Andrade"


In [ ]:
'''
Exemplo
Listar os modelos de carros alugados por cada cliente:
'''
consulta('''
SELECT cliente.nome, carro.modelo
FROM cliente
JOIN aluguel ON cliente.codcliente = aluguel.codcliente
JOIN carro ON carro.codcarro = aluguel.codcarro;
''',''' ''')

,nome,modelo
0,Túlio Nascimento,Argo
1,Bruna Pereira,Ka
2,Bruna Pereira,Ka
3,Bruna Pereira,Onix
4,Ana Silva,Polo
5,Ana Silva,Polo
6,Ana Silva,Ka
7,Lúcia Andrade,Argo
8,Lúcia Andrade,Argo
9,Túlio Nascimento,Ka


In [ ]:
'''
USAR GROUP_CONCAT para fazer com que os nomes da consulta anterior
apraceram em uma única linha
'''
consulta('''
SELECT cliente.nome, GROUP_CONCAT(carro.modelo SEPARATOR ', ') AS modelos
FROM cliente
JOIN aluguel ON cliente.codcliente = aluguel.codcliente
JOIN carro ON aluguel.codcarro = carro.codcarro
GROUP BY cliente.nome;
''',''' ''')

,nome,modelos
0,Ana Silva,"Polo, Polo, Ka"
1,Bruna Pereira,"Ka, Ka, Onix"
2,Lúcia Andrade,"Argo, Argo"
3,Túlio Nascimento,"Argo, Ka"


In [ ]:
'''
Mostre as marcas dos carros alugados por cada cliente.
'''
'''
Exemplo
Listar os modelos de carros alugados por cada cliente:
'''
consulta('''
SELECT cliente.nome, marca.marca
FROM aluguel
JOIN carro ON aluguel.codcarro = carro.codcarro
JOIN marca ON marca.codmarca = carro.codmarca
JOIN cliente ON aluguel.codcliente = cliente.codcliente;
''',''' ''')

,nome,marca
0,Bruna Pereira,Ford
1,Bruna Pereira,Ford
2,Ana Silva,Ford
3,Túlio Nascimento,Ford
4,Túlio Nascimento,Fiat
5,Lúcia Andrade,Fiat
6,Lúcia Andrade,Fiat
7,Bruna Pereira,Chevrolet
8,Ana Silva,Volkswagen
9,Ana Silva,Volkswagen


In [ ]:
consulta('''
SELECT cliente.nome, GROUP_CONCAT(marca.marca SEPARATOR ', ') AS marcas_alugadas
FROM aluguel
JOIN carro ON aluguel.codcarro = carro.codcarro
JOIN marca ON carro.codmarca = marca.codmarca
JOIN cliente ON aluguel.codcliente = cliente.codcliente
GROUP BY cliente.nome;
''',''' ''')

,nome,marcas_alugadas
0,Ana Silva,"Volkswagen, Volkswagen, Ford"
1,Bruna Pereira,"Ford, Ford, Chevrolet"
2,Lúcia Andrade,"Fiat, Fiat"
3,Túlio Nascimento,"Fiat, Ford"


In [ ]:
'''
Exiba os nomes dos clientes e as datas em que alugaram carros, concatenando as
datas.
'''
consulta('''
SELECT cliente.nome, GROUP_CONCAT(DATE_FORMAT(aluguel.data_aluguel, '%d-%m-%Y') SEPARATOR ', ') AS datas
FROM aluguel
JOIN cliente ON aluguel.codcliente = cliente.codcliente
GROUP BY cliente.nome;
''',''' ''')

,nome,datas
0,Ana Silva,"05-04-2023, 13-04-2023, 15-04-2023"
1,Bruna Pereira,"02-04-2023, 03-04-2023, 04-04-2023"
2,Lúcia Andrade,"19-04-2023, 21-04-2023"
3,Túlio Nascimento,"01-04-2023, 25-04-2023"


# VIEW

In [ ]:
consulta('''
SHOW TABLES;;
''',''' ''')

,Tables_in_locadora
0,aluguel
1,carro
2,cliente
3,marca


In [ ]:
'''
Uma view é uma consulta que fica armazenada no banco de dados como se
fosse uma tabela

Exemplo:

Crie uma consulta que exiba modelo e valor dos carros
'''
consulta('''
SELECT modelo, valor
FROM carro;
''',''' ''')


,modelo,valor
0,Ka,100.0
1,Argo,150.0
2,Onix,170.0
3,Polo,150.0
4,Kwid,120.0


In [ ]:
consulta('''
SHOW TABLES;;
''',''' ''')

,Tables_in_locadora
0,aluguel
1,carro
2,cliente
3,marca


In [ ]:
'''
Crie uma view que armazena a consulta de modelo e valor como tabela no
banco de dados
'''
consulta('''
CREATE VIEW modelo_valor AS
SELECT modelo, valor
FROM carro;
''',''' ''')

""


In [ ]:
consulta('''
SHOW TABLES;;
''',''' ''')

,Tables_in_locadora
0,aluguel
1,carro
2,cliente
3,marca
4,modelo_valor


In [ ]:
# Visualizando o schema de uma view
consulta('''
DESC modelo_valor;
''',''' ''')

,Field,Type,Null,Key,Default,Extra
0,modelo,varchar(30),NO,,None,
1,valor,float,YES,,None,


In [ ]:
# Exibindo os resultado de uma view
consulta('''
SELECT valor
FROM modelo_valor;
''',''' ''')

,valor
0,100.0
1,150.0
2,170.0
3,150.0
4,120.0


In [ ]:
'''
Considerando que a view modelo_valor já existe
tente criá-la novamente
'''
consulta('''
CREATE VIEW modelo_valor AS
SELECT modelo, valor
FROM carro;
''',''' ''')

ProgrammingError: 1050 (42S01): Table 'modelo_valor' already exists

In [ ]:
'''
No bloco anterior o erro aconteceu, pois, a view já existia,
porém se desjarmos sobrescreve-lá o código ficaria assim:
'''
consulta('''
CREATE OR REPLACE VIEW modelo_valor AS
SELECT modelo, valor
FROM carro;
''',''' ''')

""


In [ ]:
# Criando uma view juntando todos os elementos das tabelas carro e aluguel
consulta('''
CREATE OR REPLACE VIEW carro_aluguel AS
SELECT carro.*, aluguel.codaluguel, aluguel.codcliente, aluguel.data_aluguel
FROM carro
LEFT JOIN aluguel
ON carro.codcarro = aluguel.codcarro;
''',''' ''')

""


In [ ]:
# Exibindo os valores da view carro_aluguel
consulta('''
SELECT *
FROM carro_aluguel;
''',''' ''')

,codcarro,codmarca,modelo,valor,codaluguel,codcliente,data_aluguel
0,1,1,Ka,100.0,2.0,2.0,2023-04-02
1,1,1,Ka,100.0,3.0,2.0,2023-04-03
2,1,1,Ka,100.0,7.0,1.0,2023-04-15
3,1,1,Ka,100.0,10.0,3.0,2023-04-25
4,2,2,Argo,150.0,1.0,3.0,2023-04-01
5,2,2,Argo,150.0,8.0,5.0,2023-04-19
6,2,2,Argo,150.0,9.0,5.0,2023-04-21
7,3,3,Onix,170.0,4.0,2.0,2023-04-04
8,4,4,Polo,150.0,5.0,1.0,2023-04-05
9,4,4,Polo,150.0,6.0,1.0,2023-04-13


In [ ]:
'''
Utilizando a tabela cliente e a view carro_aluguel
faça uma consulta que exiba nome, modelo e data_aluguel
'''
consulta('''
SELECT cliente.nome, carro_aluguel.modelo, carro_aluguel.data_aluguel
FROM carro_aluguel
JOIN cliente ON carro_aluguel.codcliente = cliente.codcliente;
''',''',''')

,nome,modelo,data_aluguel
0,Túlio Nascimento,Argo,2023-04-01
1,Bruna Pereira,Ka,2023-04-02
2,Bruna Pereira,Ka,2023-04-03
3,Bruna Pereira,Onix,2023-04-04
4,Ana Silva,Polo,2023-04-05
5,Ana Silva,Polo,2023-04-13
6,Ana Silva,Ka,2023-04-15
7,Lúcia Andrade,Argo,2023-04-19
8,Lúcia Andrade,Argo,2023-04-21
9,Túlio Nascimento,Ka,2023-04-25


In [ ]:
'''
Utilizando a tabela cliente e a view carro_aluguel
faça uma consulta que exiba nome, modelo e data_aluguel
'''
consulta('''
SELECT cliente.nome, carro_aluguel.modelo, carro_aluguel.data_aluguel
FROM carro_aluguel
LEFT JOIN cliente ON carro_aluguel.codcliente = cliente.codcliente;
''',''',''')

,nome,modelo,data_aluguel
0,Bruna Pereira,Ka,2023-04-02
1,Bruna Pereira,Ka,2023-04-03
2,Ana Silva,Ka,2023-04-15
3,Túlio Nascimento,Ka,2023-04-25
4,Túlio Nascimento,Argo,2023-04-01
5,Lúcia Andrade,Argo,2023-04-19
6,Lúcia Andrade,Argo,2023-04-21
7,Bruna Pereira,Onix,2023-04-04
8,Ana Silva,Polo,2023-04-05
9,Ana Silva,Polo,2023-04-13


In [ ]:
'''
Utilizando a tabela cliente e a view carro_aluguel
faça uma consulta que exiba nome, modelo e data_aluguel
'''
consulta('''
SELECT cliente.nome, carro_aluguel.modelo, carro_aluguel.data_aluguel
FROM carro_aluguel
RIGHT JOIN cliente ON carro_aluguel.codcliente = cliente.codcliente;
''',''',''')

,nome,modelo,data_aluguel
0,Ana Silva,Polo,2023-04-05
1,Ana Silva,Polo,2023-04-13
2,Ana Silva,Ka,2023-04-15
3,Bruna Pereira,Ka,2023-04-02
4,Bruna Pereira,Ka,2023-04-03
5,Bruna Pereira,Onix,2023-04-04
6,Túlio Nascimento,Argo,2023-04-01
7,Túlio Nascimento,Ka,2023-04-25
8,Fernando Souza,None,None
9,Lúcia Andrade,Argo,2023-04-19
